<a href="https://colab.research.google.com/github/mhixtadavid/English_to_Igbo_translation/blob/main/English_Igbo_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y pyarrow
!pip install pyarrow

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [ ]:
!pip install openpyxl

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 4.1 MB/s eta 0:00:00


**LIBRARIES AND TOOLS**

In [ ]:
import os
import sys
import transformers
import tensorflow as tf
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ig"

**Loading and Splitting Dataset for English-Igbo Translation**

In [ ]:
# Load the dataset from a local CSV file
df = pd.read_excel('/content/drive/MyDrive/Translation_Project/Igbo_corpus_2.xlsx')

# Convert pandas DataFrame to a Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train (80%) and test (20%)
split_dataset = dataset.train_test_split(test_size=0.2)

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/819k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/800k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [44, 38552, 2, 114, 34, 10, 6594, 108, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[44, 38552, 2, 114, 34, 10, 6594, 108, 0], [313, 34, 476, 6594, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Ndewo, nke a bụ ahịrịokwu!"]))

{'input_ids': [[19166, 1365, 2, 19, 10, 28, 8488, 108, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


**Preprocessing Function for English-Igbo Translation Model**

In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "ig"


def preprocess_function(examples):
    inputs = examples["English"]
    targets = examples["Igbo"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Assuming 'dataset' is your original Dataset object
dataset_dict = DatasetDict({
    "train": dataset.shuffle(seed=42).select(range(int(len(dataset)*0.8))),  # 80% for training
    "test": dataset.shuffle(seed=42).select(range(int(len(dataset)*0.8), len(dataset)))  # Remaining 20% for test
})

In [ ]:
tokenized_datasets = dataset_dict['train'].map(preprocess_function, batched=True)

Map:   0%|          | 0/2560 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/294M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ig.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, epochs=5)

Epoch 1/5
160/160 [==============================] - 919s 5s/step - loss: 1.2524
Epoch 2/5
160/160 [==============================] - 851s 5s/step - loss: 0.9243
Epoch 3/5
160/160 [==============================] - 837s 5s/step - loss: 0.7371
Epoch 4/5
160/160 [==============================] - 835s 5s/step - loss: 0.5986
Epoch 5/5
160/160 [==============================] - 838s 5s/step - loss: 0.5061


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Translation_Project/translated_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[56126]], 'forced_eos_token_id': 0}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Translation_Project/translated_model/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/819k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/800k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/drive/MyDrive/Translation_Project/translated_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


**ENGLISH INPUT**

In [ ]:
input_text  = "Welcome, Kingdavid"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[56126    57  2380  2569     2   672    88 26727     0 56126]], shape=(1, 10), dtype=int32)


**IGBO TRANSLATED RESULT**

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

Nnwere Onwe, Eze Akwalite


**Evaluating Translation Model Using BLEU Metric**

In [ ]:
# Load the BLEU metric
metric = load_metric("sacrebleu")

# Perform evaluation on the test dataset
for example in test_dataset:
    inputs = tokenizer(example['English'], return_tensors="pt", truncation=True)
    output_ids = model.generate(inputs['input_ids'])
    prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Compute the metric with the predicted and reference translations
    metric.add(prediction=prediction, reference=[example['Igbo']])

# Print the evaluation result
print(metric.compute())

{'score': 79.01318112821501, 'counts': [2829, 2129, 1580, 1210], 'totals': [3267, 2627, 2060, 1671], 'precisions': [86.59320477502295, 81.04301484583175, 76.69902912621359, 72.41172950329144], 'bp': 1.0, 'sys_len': 3267, 'ref_len': 3155}
